# Лабораторная работа №2
## Феоктистова Эмма
## 2 курс, 3 группа, ФиКЛ

Обработаем сайт:

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://izvmor.ru/"

page = requests.get(url)
soup = BeautifulSoup(page.text)

In [3]:
for link in soup.find_all('a'):
    if link.get('href') != None:
        print(link.get('href'))

/component/banners/click/33
/component/banners/click/43
/component/banners/click/53
/
/novosti/natsproekty?layout=blog
/novosti/natsproekty?layout=blog
http://maski-lab.ru/?utm_medium=banner_top&utm_campaign=maskilb&utm_content=izvmor
/
/novosti
#
/novosti/proisshestviya
/novosti/obshchestvo
/novosti/politika
/novosti/ekonomika
/novosti/sport
/novosti/kultura
/novosti/nauka-i-obrazovanie
/novosti/novosti-kompaniy
/novosti/natsproekty
/stati
/reklama
/predpriyatiya
/meditsina
/k-75-letiyu-pobedy
/
/
http://maski-lab.ru/?utm_medium=banner_top&utm_campaign=maskilb&utm_content=izvmor
/component/banners/click/34
/component/banners/click/42
/component/banners/click/54
/novosti/obshchestvo/v-mordovii-intsident-menedzhment-pomogaet-reshat-problemy-lyudej
/novosti/obshchestvo/v-saranske-otkrylas-trekhmernaya-vystavka-podvig-geroev-bezymyannoj-vysoty
/novosti/obshchestvo/2-dekabrya-vypuskniki-mordovii-budut-pisat-itogovoe-sochinenie
/novosti/obshchestvo/v-tengushevskom-rajone-razobrali-svalku-na

Оставим только ссылки на новости:

In [4]:
urls = []

for link in soup.find_all('a'):
    if link.get('href') != None:
        if '/novosti' in link.get('href'):
            urls.append(link.get('href'))
urls

['/novosti/natsproekty?layout=blog',
 '/novosti/natsproekty?layout=blog',
 '/novosti',
 '/novosti/proisshestviya',
 '/novosti/obshchestvo',
 '/novosti/politika',
 '/novosti/ekonomika',
 '/novosti/sport',
 '/novosti/kultura',
 '/novosti/nauka-i-obrazovanie',
 '/novosti/novosti-kompaniy',
 '/novosti/natsproekty',
 '/novosti/obshchestvo/v-mordovii-intsident-menedzhment-pomogaet-reshat-problemy-lyudej',
 '/novosti/obshchestvo/v-saranske-otkrylas-trekhmernaya-vystavka-podvig-geroev-bezymyannoj-vysoty',
 '/novosti/obshchestvo/2-dekabrya-vypuskniki-mordovii-budut-pisat-itogovoe-sochinenie',
 '/novosti/obshchestvo/v-tengushevskom-rajone-razobrali-svalku-na-aerodrome',
 '/novosti/obshchestvo/oshibka-v-dokumentakh-ostavila-bez-yubileya-truzhenitsu-tyla-iz-mordovii',
 '/novosti/obshchestvo/v-mordovii-obyavleno-operativnoe-preduprezhdenie-2',
 '/novosti/kultura/ne-ozhidala-chto-boris-grebenshchikov-obratit-na-nas-vnimanie-ezhevika-spirkina-o-tvorchestve-mordovskoj-gruppy-oyme',
 '/novosti/obshches

Добавим к полученным ссылкам основную ссылку на сайт, чтобы получить полные ссылки:

In [5]:
full_urls = []

for u in urls:
    res = 'https://izvmor.ru' + u
    full_urls.append(res) 

full_urls

['https://izvmor.ru/novosti/natsproekty?layout=blog',
 'https://izvmor.ru/novosti/natsproekty?layout=blog',
 'https://izvmor.ru/novosti',
 'https://izvmor.ru/novosti/proisshestviya',
 'https://izvmor.ru/novosti/obshchestvo',
 'https://izvmor.ru/novosti/politika',
 'https://izvmor.ru/novosti/ekonomika',
 'https://izvmor.ru/novosti/sport',
 'https://izvmor.ru/novosti/kultura',
 'https://izvmor.ru/novosti/nauka-i-obrazovanie',
 'https://izvmor.ru/novosti/novosti-kompaniy',
 'https://izvmor.ru/novosti/natsproekty',
 'https://izvmor.ru/novosti/obshchestvo/v-mordovii-intsident-menedzhment-pomogaet-reshat-problemy-lyudej',
 'https://izvmor.ru/novosti/obshchestvo/v-saranske-otkrylas-trekhmernaya-vystavka-podvig-geroev-bezymyannoj-vysoty',
 'https://izvmor.ru/novosti/obshchestvo/2-dekabrya-vypuskniki-mordovii-budut-pisat-itogovoe-sochinenie',
 'https://izvmor.ru/novosti/obshchestvo/v-tengushevskom-rajone-razobrali-svalku-na-aerodrome',
 'https://izvmor.ru/novosti/obshchestvo/oshibka-v-dokumenta

Разделим ссылки на новости и разделы с новостями. Для этого воспользуемся регулярными выражениями:

In [6]:
import re
pattern = 'https://izvmor.ru/novosti/[a-z-?=]+$'
sections = []
news = []

for u in full_urls:
    if re.match(pattern, u):
        if u not in sections:
            sections.append(u)
    else:
        if u not in news and u != 'https://izvmor.ru/novosti':
            news.append(u)  
news

['https://izvmor.ru/novosti/obshchestvo/v-mordovii-intsident-menedzhment-pomogaet-reshat-problemy-lyudej',
 'https://izvmor.ru/novosti/obshchestvo/v-saranske-otkrylas-trekhmernaya-vystavka-podvig-geroev-bezymyannoj-vysoty',
 'https://izvmor.ru/novosti/obshchestvo/2-dekabrya-vypuskniki-mordovii-budut-pisat-itogovoe-sochinenie',
 'https://izvmor.ru/novosti/obshchestvo/v-tengushevskom-rajone-razobrali-svalku-na-aerodrome',
 'https://izvmor.ru/novosti/obshchestvo/oshibka-v-dokumentakh-ostavila-bez-yubileya-truzhenitsu-tyla-iz-mordovii',
 'https://izvmor.ru/novosti/obshchestvo/v-mordovii-obyavleno-operativnoe-preduprezhdenie-2',
 'https://izvmor.ru/novosti/kultura/ne-ozhidala-chto-boris-grebenshchikov-obratit-na-nas-vnimanie-ezhevika-spirkina-o-tvorchestve-mordovskoj-gruppy-oyme',
 'https://izvmor.ru/novosti/obshchestvo/mchs-mordovii-napominaet-o-shtrafakh-za-razvedenie-kostrov',
 'https://izvmor.ru/novosti/proisshestviya/sutki-v-ognennom-koltse-proveli-zhiteli-neskolkikh-sel-torbeevskogo-r

Напишем функцию, которая вынимает из html-кода нужные нам данные:

In [7]:
def GetNews(url0):
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sourse = soup0.find('meta', {'property' : 'og:site_name'}).attrs['content']
    date = soup0.find('meta', {'property': 'article:published_time'}).attrs['content']
    author = 'No author'
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    text_list = soup0.find_all('p', {'class' : None})
    text = [t.text for t in text_list]
    final_text = ' '.join(text)
    final_text = final_text.replace('\n', ' ')
    cleaned_text = final_text.split('izv_mor@moris.ru ')[1].split(' Все права на материалы')[0]
    
    return url0, sourse, date, author, title, cleaned_text


In [8]:
news1 = GetNews(news[5])
news1

('https://izvmor.ru/novosti/obshchestvo/v-mordovii-obyavleno-operativnoe-preduprezhdenie-2',
 'Новостной портал Мордовии',
 '2020-10-17T10:57:08+03:00',
 'No author',
 'В Мордовии объявлено оперативное предупреждение',
 ' МЧС региона объявило оперативное предупреждение на 18 октября. В ведомстве сообщают, что по прогнозам метеорологов завтра по республике возможны порывы юго-восточного ветра до 17 метров в секунду. В ведомстве призывают соблюдать все меры безопасности и советуют убрать со двора или балкона вещи, автомобили лучше поставить в гараж, если его нет, то машину следует парковать вдали от деревьев и рекламных конструкций. На улице обходите рекламные щиты и сооружения повышенного риска - мосты, эстакады, линии электропередач. Если вы оказались в беде, звоните по номеру 112.    ')

Создадим файл, в который будем вписывать данные про каждую новость, полученные с помощью функции выше:

In [9]:
with open('lab_news.txt', 'w', encoding='utf-8') as file:
    for n in news:
        res = GetNews(n)
        file.write('=====')
        file.write('\n')
        for i in res:
            file.write(i)
            file.write('\n')

Файл, который показывает, в какой секции находится новость:

In [18]:
with open('lab_news_graph.txt', 'w', encoding='utf-8') as file:
    for s in sections:
        for n in news:
            if n.startswith(s):
                file.write('===== \n')
                file.write(s)
                file.write('\n')
                res = GetNews(n)
                file.write('=====')
                file.write('\n')
                for i in res:
                    file.write(i)
                    file.write('\n')

Функции для создания двоичных деревьев:

In [11]:
def addValueToNode(node, value):
    if node[0] == value:
        return

    if value < node[0]:
        if node[1] == None:
            node[1] = [value, None, None]
        else:
            addValueToNode(node[1], value)

    else:
        if node[2] == None:
            node[2] = [value, None, None]
        else:
            addValueToNode(node[2], value)
            
    
def addValueToTree(root, value):
    if root == None or root == []:
        return [value, None, None]

    addValueToNode(root, value)
    return root

def printTreeDepth(node, right, left=0, strings=None, layer=0):
    if strings == None:
        strings = []

    if len(strings) <= layer:
        strings.append(' ' * left)

    elif len(strings[layer]) < left:
        strings[layer] += ' ' * (left - len(strings[layer]))

    dif1 = int((right - left - len(str(node[0]))) / 2) 
    dif2 = right - left - len(str(node[0])) - dif1

    strings[layer] += ' ' * dif1 + str(node[0]) + ' ' * dif2

    c = int((right + left) / 2)
    if node[1] != None:
        strings = printTreeDepth(node[1], c, left, strings, layer+1)
    if node[2] != None:
        strings = printTreeDepth(node[2], right, c+1, strings, layer+1)
        
    if layer == 0:
        for s in strings:
            print('\n', s)        
    return strings

def printTreeWidth2(nodes, width):
    if len(nodes) == 0:
        return

    woe = int(width / len(nodes))
    s = ''
    nodes2 = []
    for node in nodes:
        if node == None:
            s += ' ' * woe
            nodes2.extend([None, None])
        else:
            dif1 = int((woe - len(node[0])) / 2)
            dif2 = woe - len(node[0]) - dif1
            s += ' ' * dif1 + str(node[0]) + ' ' * dif2
            nodes2.append(node[1])
            nodes2.append(node[2])
    strings = [s]

    if any([n != None for n in nodes2]):
        strings.extend(printTreeWidth2(nodes2, width))
        
    return strings
    

def printTreeWidth(tree, width):
    strings = printTreeWidth2([tree], width)
    for s in strings:
        print('\n'+s)

printTree = printTreeDepth

Представим отсортированный текст каждой новости в виде двоичного дерева, добавим это дерево в общий список:

In [12]:
trees = []
for n in news:
    res = GetNews(n)
    text = res[5]
    words = text.split()
    sorted(words)
    tree = None
    for i in words:
        tree = addValueToTree(tree, i)
        trees.append(tree)

print(trees[0])
printTree(trees[0], 80)

['За', ['2020', ['(районными', ['(из', None, None], ['2019', ['178,', ['158,', ['111,', ['106.', None, None], None], None], None], None]], ['«Инцидент', ['7866', ['3234', ['301.', None, ['315,', None, None]], ['6870', ['6826;', ['387,', ['347,', None, None], ['5224.', None, ['588,', None, None]]], None], None]], ['VKontakte', ['Instagram', ['FB', None, None], None], None]], ['Для', ['В', ['Ардатовском', None, None], None], None]]], ['три', ['квартала', ['года', ['власти', ['Мордовии', ['Из', ['Зубово-Полянском', None, None], ['Ковылкинском', ['Инсарском', None, None], ['Лямбирском', None, None]]], ['администрациями,', ['Число', ['Наибольшее', None, ['Одноклассники', None, ['Твиттера', ['Саранск', ['Рузаевском', None, None], None], ['Темниковском', None, None]]]], None], ['ведомствами)', ['благодаря', ['аналитической', None, None], ['было', None, ['в', None, None]]], None]]], ['выросло,', ['втором', None, None], None]], ['государственной', ['городском', None, None], ['и', ['декабрь', No

['                                       За                                       ',
 '                  2020                                     три                  ',
 '     (районными           «Инцидент           квартала                –         ',
 '   (из       2019      7866       Для      года    представителями января  ',
 '           178,      3234 VKontakte В    властигосударственнойоргановсамоуправлениячисло',
 '           158,      301.6870InstagramАрдатовскомМордовиивыросло,городскомиместногопорассмотреносистеме',
 '           111,        315,6826;FB       Изадминистрациями,второмдекабрьинцидентовменеджмент»министерствамиотпришлосьссвязитретий',
 '           106.         387,             Зубово-ПолянскомКовылкинскомЧисловедомствами)запросовинцидента).кварталмаксимальноеместенихпервуюприходитрайонесетисравнения,третьем',
 '                        347,5224.         ИнсарскомЛямбирскомНаибольшееблагодаряжителейзафиксированоизинцидентов.количествонаокругеочередь,соцсетях',


Функция для индексации:

In [13]:
def bubbleSortIndex(a, N):
    index = list(range(N))
    for i in range(N-1):
        for j in range(N-i-1):
            if a[index[j]] > a[index[j+1]]:
                index[j], index[j+1] = index[j+1], index[j]
    return index

Сделаем список индексов для текста каждой новости:

In [14]:
indexes = []
for n in news:
    res = GetNews(n)
    text = res[5]
    words = text.split()
    sorted(words)
    index = bubbleSortIndex(words, len(words))
    indexes.append(index)

print(indexes[0])

[28, 13, 135, 130, 126, 122, 45, 3, 86, 117, 37, 112, 81, 103, 108, 75, 49, 26, 88, 84, 73, 21, 124, 104, 39, 0, 119, 87, 128, 110, 114, 12, 66, 94, 79, 105, 101, 90, 132, 51, 14, 19, 18, 24, 47, 55, 58, 61, 98, 109, 113, 118, 123, 127, 131, 17, 8, 77, 57, 4, 46, 99, 7, 44, 54, 52, 97, 9, 16, 89, 70, 38, 27, 96, 50, 93, 35, 2, 31, 67, 95, 65, 30, 23, 78, 10, 15, 33, 76, 82, 91, 29, 100, 6, 53, 60, 64, 59, 43, 5, 69, 32, 106, 115, 120, 133, 25, 48, 41, 63, 11, 62, 72, 20, 68, 71, 56, 40, 34, 83, 1, 92, 42, 22, 36, 74, 80, 85, 102, 107, 111, 116, 121, 125, 129, 134]
